In [1]:
!rm -rf mnist_data
!mkdir mnist_data
%cd mnist_data

!curl -O http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
!curl -O http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
!curl -O http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
!curl -O http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz

!gunzip *.gz
%cd ..

/Users/akshatgupta/Desktop/IIITD/SEM_6/DL/DL-Assignments/Assignment1/mnist_data
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9680k  100 9680k    0     0  4602k      0  0:00:02  0:00:02 --:--:-- 4602k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 28881  100 28881    0     0  70208      0 --:--:-- --:--:-- --:--:-- 70099
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1610k  100 1610k    0     0   634k      0  0:00:02  0:00:02 --:--:--  634k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4542  100  4542    0     0   7668      0 --:--

In [2]:
import torch

def tensor_transform(data):
    return torch.from_numpy(data).to(torch.float32)

In [3]:
import os, struct
import numpy as np


class CustomDataset:
    def __init__(self,root,train=True):
        """
        Args:
            root (str): path to the directory of the dataset
            train (bool, default=True): if True return training data, else return testing data
        """
        self.root = root
        self.train = train

        self.data_name = ('train-images-idx3-ubyte' if train else 't10k-images-idx3-ubyte')
        self.labels_name = ('train-labels-idx1-ubyte' if train else 't10k-labels-idx1-ubyte')
        self.data, self.labels = self.load_data()


    def load_data(self):
        data_path = os.path.join(self.root, self.data_name)
        labels_path = os.path.join(self.root, self.labels_name)

        with open(data_path, 'rb') as f:
            _, _, rows, cols = struct.unpack('>IIII', f.read(16))
            data = np.frombuffer(f.read(), dtype=np.uint8)
            data = data.reshape(-1, rows, cols)
        with open(labels_path, 'rb') as f:
            _, _ = struct.unpack('>II', f.read(8))
            labels = np.frombuffer(f.read(), dtype=np.uint8)

        return data, labels


    def __len__(self):
        return len(self.data)


    def __getitem__(self,index):
        img, target = list(self.data[index].reshape(1,28,28)), int(self.labels[index])
        return img, target

In [4]:
class CustomDataLoader:
    def __init__(self,dataset,batch_size=64,shuffle=False,transform=None):
        """
        Args:
            dataset (obj): Dataset object that will be loaded into the stream; must implement __getitem__ and __len__ functions
            batch_size (int, default=64): batch size
            shuffle (bool, default=False): if True shuffles the data before loading into stream
            transform (callable, default=None): transform function to apply to data
        """
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.transform = transform
        self.num_samples = len(dataset)
        self.num_batches = (self.num_samples + batch_size - 1) // batch_size

    def __iter__(self):
        if self.shuffle:
            indices = torch.randperm(self.num_samples)
        else:
            indices = torch.arange(self.num_samples)

        for i in range(0, self.num_samples, self.batch_size):
            batch_indices = indices[i:min(self.num_samples,i+self.batch_size)]
            batch = [self.dataset[index] for index in batch_indices]
            batch = tuple(zip(*batch))
            data, label = batch

            if self.transform is not None:
                data = self.transform(np.array(data))

            yield {'data':data,'label':label}

    def __len__(self):
        return self.num_batches

In [5]:
data = CustomDataset(
    root="./mnist_data/",
    train=True
)

In [6]:
dataloader = CustomDataLoader(
    dataset=data,
    batch_size=128,
    shuffle=False,
    transform=tensor_transform
)

In [8]:
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms


class MNISTDataset(Dataset):
    def __init__(self, root, transform=None):
        self.dataset = datasets.MNIST(root=root, train=True, transform=transform, download=True)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        return {'data': image, 'label': label}

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [9]:
transform = transforms.Compose([transforms.ToTensor(),])
mnist_data = MNISTDataset(root="./mnist_data/",transform=transform)

100%|██████████| 9912422/9912422 [00:02<00:00, 3880066.81it/s]


Extracting ./mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 1073336.59it/s]


Extracting ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:01<00:00, 1108106.38it/s]


Extracting ./mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 3435622.86it/s]

Extracting ./mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw



In [11]:
batch_size = 128
mnist_dataloader = DataLoader(mnist_data,batch_size=batch_size,shuffle=False)

In [ ]:
import time

